# Introduction
As we are all painfully aware, airline delays are all too common of an experience.  And as airports are one of the most resource intensive shared spaces in modern commerce, delays bring increased costs to crews, airport personel, limited gate availability, fuel and maintenance cost as well as to the passengers that can easily total more than $30 billion per year.  While many of the event sequences that cause these delays are too complicated to fully prevent, predicting them and providing advanced notice to the multitude of interested parties would not only be a convienece, but a siginfcant savings in wasted resources.  Passengers would not have to spend as much time occupying expensive share resources and airport personnel to could take the necessary steps to realocate limited resources such as gates and scheduled equipment usage to not only save resources, but mitigate the propagation of the delay downstream to additional flights, crews and passengers.

## Decision Trees
Decisions trees are a classification tool which takes its name from the tree-like structure it forms where each node is a test on a feature and each branch divides the dataset on the outcome of that test.  We found that decision trees have a number of qualities which are particulary suited for our task.  A few of the most promenant are:
* They are interpertable since each node can be phrased as a question that tests one of the features.
* They naturally express feature importance.  Since we have hundreds of potential features, many of which may have undesirable qualities such as colinearity, this model provides an approach for sensible feature removal.
* They can are robust to unseen categorical values.  We found that over time different airports and weather station come online, and in larger time scales, the airlines themselves are reorganized as businesses, and any of these events can cause a change in the categorical values our model receives.
* They are resilant to missing values.  We also found that many of the weather measurements are inconsistent or unusable, and although we are putting significant effort into filling those values, it warrants forseeing that there may be intances of missing values that will still want to predict against in the future.
* Of course, since the volume of data we're training against is so large, the fact that random forests are constructed from indepented trees which can be built in parallel is an important performance consideration.

Our original baseline model was built using linear regression, which also has very nice qualities in regards to interpertability, idenification of feature performance and parallel implementation. However would also have to invest additional resources in preprocessing and creating deterministic outcomes for changes in categorical variables or unexpected missing variable that would potentially take away from better feature engineering and feature selection, as well as set us up for a less robust inference unit production.

## Nearest Station Neighbors
One of the siginficant challenges with the dataset is that the weather metrics reported from the stations located at the airports is quite frequently missing or unusable.  One remedy we sought for this was to look to nearby stations, since weather is spatially localized, and at the speed planes travel at there is a considerable radius of weather conditions relavent to landing and takeoff operations.  The challenge is that there are many thousands of weather stations, and finding all of the stations that are within the range of interest for each airport would involve a very large number of possible permutations of stations.  Fortunately this can be formulated as an embarassingly parallel problem suitable for the MapReduce framework.  The distance calculation between each station performed by the haversine formula, which only depends on the coordinates of the two stations which the distance is being measured between, so it can be evenly spread across as many mappers as available.  The resulting distance is emitted with a compound key consisting of the airport's station identifier, followwed by the target weather station's.  The shuffle stage then naturally brings each airport's neighbors onto the same partiton and orders them by the closest distance, which is exactly the order in which we want to try to fill in missing or invalid weather measurements.

## Pagerank
We are interested in engineering features that will help our model predict how flight delays propagate through the airport network causing additional downstream delays.  To this end, the Pagerank algorithm